In [ ]:
import csv
import numpy as np
import open3d as o3d
from matplotlib import pyplot as plt 
from sklearn.neighbors import KDTree
from sklearn.decomposition import PCA
import plotly.graph_objects as go
import ipywidgets as widgets # for interactive sliders
import pickle
from timeit import timeit
from scipy.spatial import cKDTree

from itertools import product


%load_ext autoreload 
%autoreload 2
from digiforest_analysis.tasks.tree_reconstruction import Tree, Circle
from digiforest_analysis.utils.timing import Timer
timer = Timer()

In [ ]:
DATASET_DIR = "/home/ori/git/realtime-trees/single_trees/clustering_2/"
SLICE_HEIGHT = 0.5
TREE_ID = 63

In [ ]:
file_name = DATASET_DIR + "tree" + str(TREE_ID).zfill(3) + ".pkl"
with open(file_name, 'rb') as file:
    cluster = pickle.load(file)

In [ ]:
slice_height = 2.0
slice_thickness = 2.0
cloud = cluster['cloud'].point.positions.numpy()
slice = cloud[np.logical_and(cloud[:, 2] >= slice_height - slice_thickness / 2, cloud[:, 2] < slice_height + slice_thickness / 2,)][:, :2]
print(len(slice))

# timing_hough = timeit("Circle.from_cloud_hough(slice)", "from __main__ import Circle, slice", number=1000)
# print(f"Hough algo took {timing_hough:.3f} ms")
# timing_new = timeit("new_hough(slice)", "from __main__ import new_hough, slice", number=1000)
# print(f"New algo took {timing_new:.3f} ms")

with timer("OLD"):
    circ_hough = Circle.from_cloud_hough(slice)
with timer("NEW"):
    # circ_new, circles = Circle.from_cloud_ransahc(slice, min_radius=0.05, max_radius=0.5, sampling="weighted", max_points=100)
    circ_new = Circle.from_cloud_ransahc(slice, min_radius=0.05, max_radius=0.5, sampling="weighted", max_circles=500)

print(timer)
plt.scatter(slice[:, 0], slice[:, 1], s=5)
plt.gca().set_aspect('equal', adjustable='box')
# plot circle
plt.gca().add_artist(plt.Circle((circ_hough.x, circ_hough.y), circ_hough.radius, color='r', fill=False))
plt.gca().add_artist(plt.Circle((circ_new.x, circ_new.y), circ_new.radius, color='g', fill=False))
# plt.scatter(circles[:, 0], circles[:, 1], s=5, color='r')

In [ ]:
import colorsys
import numpy as np
from matplotlib import pyplot as plt
import ipywidgets as widgets
import pickle, os

%load_ext autoreload 
%autoreload 2
from digiforest_analysis.tasks.tree_reconstruction import Tree, Circle
# %matplotlib notebook

path = "/home/ori/git/digiforest_drs/trees/logs/raw"
TREE_ID = 12
tree_path = os.path.join(path, f"tree{TREE_ID:03d}.pkl")
with open(tree_path, 'rb') as file:
    tree : Tree = pickle.load(file)

def align_clouds(clusters):
    trafos_map = [c["info"]["T_sensor2map"] @ c["info"]["axis"]["transform"] for c in clusters]
    clouds_map = [c["cloud"].point.positions.numpy() @ c["info"]["T_sensor2map"][:3, :3].T + c["info"]["T_sensor2map"][:3, 3] for c in clusters]
    mean_position = np.mean([t[:3, 3] for t in trafos_map], axis=0)
    deltas = [t[:3, 3] - mean_position for t in trafos_map]
    deltas = [np.array([d[0], d[1], 0]) for d in deltas]
    return [c - d for c, d in zip(clouds_map, deltas)]


# make a 2d plot with a slider that changes the height of the slice
clouds = [
    cluster['cloud'].point.positions.numpy() @ cluster['info']['T_sensor2map'][:3, :3].T + cluster['info']['T_sensor2map'][:3, 3]
    for cluster in tree.clusters
]
clouds_aligned = align_clouds(tree.clusters)
cloud_merged = np.concatenate(clouds, axis=0)
cloud_aligned_merged = np.concatenate(clouds_aligned, axis=0)
# cloud_aligned_merged = tree.points
hues = np.linspace(0, 1, len(clouds) + 1)[:len(clouds)]
colors = [colorsys.hsv_to_rgb(h, 1, 1) for h in hues]

slice_thickness = 0.2
# min and max width and height are 10th and 90 th percentiles
min_width, max_width = np.percentile(cloud_merged[:, 0], [2, 98])
min_height, max_height = np.percentile(cloud_merged[:, 1], [2, 98])
@widgets.interact(height=widgets.FloatSlider(min=cloud_merged[:, 2].min(), max=cloud_merged[:, 2].max(), step=0.3, value=0.0))
def update_slice_height(height):
    fig, ax = plt.subplots(1, 3, figsize=(15, 5))
    for a in ax:
        a.set_aspect('equal', adjustable='box')
        a.set_xlim(min_width, max_width)
        a.set_ylim(min_height, max_height)
    for cloud, color in zip(clouds, colors):
        slice = cloud[np.logical_and(cloud[:, 2] >= height - slice_thickness / 2, cloud[:, 2] < height + slice_thickness / 2,)][:, :2]
        ax[0].scatter(slice[:, 0], slice[:, 1], s=1, color=color)
    for cloud, color in zip(clouds_aligned, colors):
        slice = cloud[np.logical_and(cloud[:, 2] >= height - slice_thickness / 2, cloud[:, 2] < height + slice_thickness / 2,)][:, :2]
        ax[1].scatter(slice[:, 0], slice[:, 1], s=1, color=color)
    slice_merged = cloud_merged[np.logical_and(cloud_merged[:, 2] >= height - slice_thickness / 2, cloud_merged[:, 2] < height + slice_thickness / 2,)][:, :2]
    slice_aligned_merged = cloud_aligned_merged[np.logical_and(cloud_aligned_merged[:, 2] >= height - slice_thickness / 2, cloud_aligned_merged[:, 2] < height + slice_thickness / 2,)][:, :2]
    ax[2].scatter(slice_aligned_merged[:, 0], slice_aligned_merged[:, 1], s=1, color='k')
    # fit hough circle
    center_region = Circle(tree.axis["transform"][:3, 3], tree.axis["radius"])
    min_radius = 0.5 * tree.axis["radius"]
    max_radius = 1.5 * tree.axis["radius"]
    circ_hough = Circle.from_cloud_hough(slice_merged, min_radius=0.05, max_radius=0.5)
    circ_hough_aligned = Circle.from_cloud_hough(slice_aligned_merged, min_radius=0.05, max_radius=0.5)
    circ_ransahc = Circle.from_cloud_ransahc(slice_merged, min_radius=min_radius, max_radius=max_radius, center_region=center_region)
    circ_ransahc_aligned = Circle.from_cloud_ransahc(slice_aligned_merged, min_radius=min_radius, max_radius=max_radius, center_region=center_region)
    ax[0].add_artist(plt.Circle((circ_hough.x, circ_hough.y), circ_hough.radius, color='r', fill=False, linewidth=2))
    ax[0].add_artist(plt.Circle((circ_ransahc.x, circ_ransahc.y), circ_ransahc.radius, color='g', fill=False, linewidth=2))
    ax[2].add_artist(plt.Circle((circ_hough_aligned.x, circ_hough_aligned.y), circ_hough_aligned.radius, color='r', fill=False, linewidth=2))
    ax[2].add_artist(plt.Circle((circ_ransahc_aligned.x, circ_ransahc_aligned.y), circ_ransahc_aligned.radius, color='g', fill=False, linewidth=2))

In [ ]:
import open3d as o3d
import numpy as np
import colorsys
import pickle, os
from digiforest_analysis.tasks.tree_reconstruction import Tree, Circle

path = "/home/ori/git/digiforest_drs/trees/logs/raw"
TREE_ID = 50
tree_path = os.path.join(path, f"tree{TREE_ID:03d}.pkl")
with open(tree_path, 'rb') as file:
    tree : Tree = pickle.load(file) 

# clouds = [cluster['cloud'].transform(cluster['info']['T_sensor2map']).point.positions.numpy() for cluster in tree.clusters]
clouds = [cluster['cloud'].point.positions.numpy() @ cluster['info']['T_sensor2map'][:3, :3].T + cluster['info']['T_sensor2map'][:3, 3]for cluster in tree.clusters]
hues = np.linspace(0, 1, len(clouds) + 1) [:len(clouds)]
colors = np.concatenate([np.array([[colorsys.hls_to_rgb(h, 0.6, 1)]]*len(c)) for h, c in zip(hues, clouds)]).squeeze()
# for cluster in tree.clusters:
#     axis_trafo = cluster["info"]['T_sensor2map'] @ cluster["info"]["axis"]["transform"]
#     circle_lower = Circle(axis_trafo[:3, 3], cluster["info"]["axis"]["radius"], axis_trafo[:3, 2])
#     circle_lower = Circle(axis_trafo[:3, 3] + axis_trafo[:3, 2] * cluster)
#     verts, tris = circle_lower.generate_cone_frustum_mesh(circle_upper)
#     verts_vec = o3d.utility.Vector3dVector(verts)
#     tris_vec = o3d.utility.Vector3iVector(
#         np.concatenate((tris, np.flip(tris, axis=1)), axis=0)
#     )
#     terrain_mesh = o3d.geometry.TriangleMesh(verts_vec, tris_vec)
# raise ValueError
cloud_aligned = tree.points
o3d_cloud = o3d.t.geometry.PointCloud(np.vstack(clouds))
o3d_cloud_aligned = o3d.t.geometry.PointCloud(cloud_aligned)
o3d_cloud.point.colors = colors
o3d_cloud_aligned.point.colors = colors

# min_height = 5
# max_height = 6
# mask = np.logical_and(o3d_cloud.point.positions.numpy()[:, 2] > min_height, o3d_cloud.point.positions.numpy()[:, 2] < max_height)
# o3d_cloud = o3d_cloud.select_by_mask(mask)
# mask = np.logical_and(o3d_cloud_aligned.point.positions.numpy()[:, 2] > min_height, o3d_cloud_aligned.point.positions.numpy()[:, 2] < max_height)
# o3d_cloud_aligned = o3d_cloud_aligned.select_by_mask(mask)

aligned_cloud_flag = False

def toggle_point_cloud(vis):
    global aligned_cloud_flag
    current_view = vis.get_view_control().convert_to_pinhole_camera_parameters()
    
    if aligned_cloud_flag:
        vis.clear_geometries()
        vis.add_geometry(o3d_cloud_aligned.to_legacy())
        aligned_cloud_flag = False
        print("Showing Aligned Cloud")
    else:
        vis.clear_geometries()
        vis.add_geometry(o3d_cloud.to_legacy())
        print("Showing Non-Aligned Cloud")
        aligned_cloud_flag = True
    vis.get_view_control().convert_from_pinhole_camera_parameters(current_view, True)

visualizer = o3d.visualization.VisualizerWithKeyCallback()
visualizer.create_window()
visualizer.add_geometry(o3d_cloud.to_legacy())
visualizer.register_key_callback(ord("C"), toggle_point_cloud)
visualizer.run()
visualizer.destroy_window()

In [ ]:
import pickle, os, time
%load_ext autoreload
%autoreload 2
from digiforest_analysis.tasks.tree_reconstruction import Tree, Circle

path = "/home/ori/git/digiforest_drs/trees/logs/raw"
TREE_ID = 63
tree_path = os.path.join(path, f"tree{TREE_ID:03d}.pkl")
with open(tree_path, 'rb') as file:
    tree : Tree = pickle.load(file) 
TIME = time.perf_counter()
print(tree.reconstruct2(max_radius_deviation=100))
print(time.perf_counter() - TIME)   

# # DEBUG inside tree.reconstruct2
# from matplotlib import pyplot as plt
# import matplotlib
# import numpy as np
# import colorsys
# matplotlib.use("TKagg")
# fig, ax = plt.subplots()
# ax.set_xlabel("x")
# ax.set_ylabel("y")
# ax.set_xlim((-0.5, 0.5))
# ax.set_ylim((-0.5, 0.4))
# ax.set_title("Averaging of individual Payload Clouds")
# ax.set_aspect('equal', adjustable='box')
# hues = np.linspace(0, 1, len(ransahc_circles) + 1)[:len(ransahc_circles)]
# colors = [list(colorsys.hls_to_rgb(hue, 0.55, 0.8)) for hue in hues]
# for points, circle, color in zip(debug_slice_points, ransahc_circles, colors):
#     ax.scatter(points[:, 0], points[:, 1], c=[color]*len(points), s=2)
#     ax.add_artist(plt.Circle((circle.x, circle.y), circle.radius, color=color, fill=False))
# ax.add_artist(plt.Circle((circle.x, circle.y), circle.radius, color="g", linestyle='dashed', fill=False, linewidth=5))
# fig.show()

In [ ]:
from digiforest_analysis_ros.src.digiforest_analysis_ros.forest_analysis import TreeManager
import pickle, os

path = "/home/ori/git/digiforest_drs/trees/logs/raw_sar_exp03"
tm = TreeManager()
trees = []
for file in os.listdir(path):
    if file.endswith(".pkl"):
        with open(os.path.join(path, file), 'rb') as f:
            tree = pickle.load(f)
            trees.append(tree)
tm.trees = trees
tm.num_trees = len(trees)
# write back pickle of tm
with open(os.path.join(path, "tree_manager.pkl"), 'wb') as f:
    pickle.dump(tm, f)

# Experiments

### Data Loading

In [ ]:
import laspy
import csv, os, pickle
import open3d as o3d
import numpy as np
from scipy.interpolate import RegularGridInterpolator
o3d.utility.set_verbosity_level(o3d.utility.VerbosityLevel.Error)
from matplotlib import pyplot as plt

%load_ext autoreload
%autoreload 2
from digiforest_analysis.tasks.tree_reconstruction import Tree, Circle
from digiforest_analysis_ros.forest_analysis import TreeManager
from digiforest_analysis.tasks.terrain_fitting import TerrainFitting
from digiforest_analysis.utils.distances import distance_line_to_line

def load_tuples(data_path, tree_manager):
    april_tag_pos_path = os.path.join(data_path, "april_tag_locations.csv")
    april_tag_measurement_path = os.path.join(data_path, "forester_data.csv")
    ground_cloud_path = os.path.join(data_path, "ground_cloud.las")

    # reconstructed trees
    ours_trees = tree_manager.trees

    # ground truth trees
    id = 0
    tls_trees = []
    for file_name in os.listdir(data_path):
        if file_name.endswith(".csv") and "_result" in file_name:
            with open(os.path.join(data_path, file_name), 'r') as file:
                csv_reader = csv.DictReader(file)
                circle_stack = []
                for row in csv_reader:
                    circle = Circle(
                        np.array([
                            float(row["center_x"]),
                            float(row["center_y"]),
                            float(row["slice_height"])
                        ]), 
                        float(row["radius"]))
                    circle_stack.append(circle)
                
                tree = Tree(id)
                tree.reconstructed = True
                tree.circles = circle_stack
                transform = np.eye(4)
                dir_vec = circle_stack[2].center - circle_stack[1].center
                dir_vec /= np.linalg.norm(dir_vec)
                dir_vec_normal = np.array([-dir_vec[1], dir_vec[0], 0])
                dir_vec_normal /= np.linalg.norm(dir_vec_normal)
                transform[:3, 2] = dir_vec
                transform[:3, 0] = dir_vec_normal
                transform[:3, 1] = np.cross(dir_vec, dir_vec_normal)
                transform[:3, 3] = circle_stack[0].center
                tree.clusters = [{
                    "info": {
                        "T_sensor2map": np.eye(4),
                        "axis": {
                            "transform": transform,
                            "radius": circle_stack[0].radius
                        },
                        "file": file_name
                    }
                }]
                tls_trees.append(tree)
                id += 1

    # april tags
    with open(april_tag_pos_path, 'r') as file:
        csv_reader = csv.DictReader(file)
        april_tag_positions = {int(row["tag_id"]): np.array([float(row["x"]), float(row["y"]), float(row["z"])]) for row in csv_reader}

    # manual DBHs
    dbhs = {}
    with open(april_tag_measurement_path, 'r') as file:
        csv_reader = csv.DictReader(file)
        for row in csv_reader:
            if row["DBH [cm]"] == "":
                continue
            dbhs[int(row["Tree"])] = float(row["DBH [cm]"].replace(",", "."))

    # tree matching
    tree_tuples = []
    for i_gt, tls_tree in enumerate(tls_trees):
        result = {"tls_tree": tls_tree,}
        gt_axis = {"transform": tls_tree.axis["transform"]}
        matches = 0
        for ours_tree in ours_trees:
            ours_axis = {"transform": ours_tree.axis["transform"]}
            dist = distance_line_to_line(gt_axis, ours_axis, clip_heights=[0,10])
            if dist < 0.5:
                result["ours_tree"] = ours_tree
        for k, v in april_tag_positions.items():
            if np.linalg.norm(v[:2] - tls_tree.axis["transform"][:2, 3]) < 0.5:
                result["april_tag"] = k 
                result["manual_dbh"] = dbhs[k]
        if "ours_tree" in result:
            tree_tuples.append(result)

    print(f"Detection Precision: {len(tree_tuples) / id * 100 :.2f} %")

    terrain_cloud = laspy.read(ground_cloud_path)
    terrain_cloud = np.vstack((terrain_cloud.x, terrain_cloud.y, terrain_cloud.z)).T
    terrain_cloud = o3d.t.geometry.PointCloud(terrain_cloud)
    terrain = TerrainFitting().process(cloud=terrain_cloud)
    terrain_interpolator = RegularGridInterpolator(
        points=(terrain[:, 0, 0], terrain[0, :, 1]),
        values=terrain[:, :, 2],
        method="linear",
        bounds_error=False,
        fill_value=None
    )
    return tree_tuples, terrain, terrain_interpolator

In [ ]:
import numpy as np
import open3d as o3d

datasets = {"mixed": {}, "deciduous": {}, "conifer": {}}
paths = {
    "mixed": "/home/ori/datasets/digiforest_gt_prefor_M",
    "deciduous": "/home/ori/datasets/digiforest_gt_prefor_D",
    "conifer": "/home/ori/datasets/digiforest_gt_prefor_C"
}
OURS_COLOR = "#0173b2"
TLS_COLOR = "#de8f05"
MANUAL_COLOR = "#029e73"

MIXED_MARKER = "o"
DECIDUOUS_MARKER = "x"
CONIFER_MARKER = "^"


In [ ]:
for plot_type in datasets.keys():
    print(f"Loading {plot_type} dataset")
    with open(os.path.join(paths[plot_type], "tree_manager.pkl"), 'rb') as file:
        tm : TreeManager = pickle.load(file)
    tree_tuples, terrain, terrain_interpolator = load_tuples(paths[plot_type], tm)
    datasets[plot_type]["tuples"] = tree_tuples
    datasets[plot_type]["terrain"] = {"mesh_grid": terrain, "interpolator": terrain_interpolator}

## Experiment 01: Accuracy

### Reconstruction

In [ ]:
from time import perf_counter
from matplotlib import pyplot as plt
from tqdm.auto import tqdm
from digiforest_analysis.utils.meshing import meshgrid_to_mesh
%load_ext autoreload
%autoreload 2
%matplotlib inline
from digiforest_analysis.tasks.tree_reconstruction import Tree, Circle

selection = None
# plot_types = ["mixed", "deciduous", "conifer"]
plot_types = ["conifer"]
max_radii = {"mixed": 0.25, "deciduous": 0.15, "conifer": 0.5}
out_dir = "/home/ori/git/digiforest_drs/trees/evaluation/weighting_test"
os.makedirs(out_dir, exist_ok=True)
suffix = ""
num_runs = 1

for plot_type in plot_types:
    for i_run in range(num_runs):
        tree_tuples = datasets[plot_type]["tuples"]
        terrain = datasets[plot_type]["terrain"]["mesh_grid"]
        terrain_interpolator = datasets[plot_type]["terrain"]["interpolator"]
        
        data_path = paths[plot_type]
        if selection:
            tree_pair_iter = tqdm(tree_tuples[selection: selection+1])
        else:
            tree_pair_iter = tqdm(tree_tuples)

        manual_dbhs = []
        tls_dbhs = []
        ours_dbhs = []
        viz_objects = []
        for tree_pair in tree_pair_iter:
            ground_elevation = terrain_interpolator(tree_pair["tls_tree"].axis["transform"][:2, 3])[0]
            # manual measurement
            manual_dbhs.append(tree_pair["manual_dbh"] if "manual_dbh" in tree_pair else np.nan)
            
            # TLS measurement
            tls_tree : Tree = tree_pair["tls_tree"]
            tls_tree.compute_dbh(ground_elevation)
            tls_dbhs.append(tls_tree.dbh * 100 if tls_tree.dbh is not None else np.nan)
            
            # Ours measurement
            ours_tree : Tree = tree_pair["ours_tree"]
            time_before = perf_counter()
            if ours_tree.reconstruct3(max_radius=max_radii[plot_type]):
                reco_time = perf_counter() - time_before
                ours_tree.__dict__["reco_time"] = reco_time
                ours_tree.compute_dbh(ground_elevation)
                ours_dbhs.append(ours_tree.dbh * 100 if ours_tree.dbh is not None else np.nan)
            else:
                ours_tree.__dict__["reco_time"] = np.nan
                ours_dbhs.append(np.nan)
            
            if tls_tree.dbh is not None and ours_tree.dbh is not None:
                verts, tris = tls_tree.generate_mesh()
                mesh = o3d.geometry.TriangleMesh(o3d.utility.Vector3dVector(verts), o3d.utility.Vector3iVector(tris))
                mesh.paint_uniform_color([0.2, 0.8, 0.2])
                mesh.compute_vertex_normals()
                viz_objects.append(mesh)
                
                verts, tris = ours_tree.generate_mesh()
                mesh = o3d.geometry.TriangleMesh(o3d.utility.Vector3dVector(verts), o3d.utility.Vector3iVector(tris))
                mesh.paint_uniform_color([0.8, 0.2, 0.2])
                mesh.compute_vertex_normals()
                viz_objects.append(mesh)
                
                point_cloud = o3d.t.geometry.PointCloud(ours_tree.points)
                point_cloud.paint_uniform_color([0, 0, 1])
                point_cloud = point_cloud.to_legacy()
                viz_objects.append(point_cloud)

        verts, tris = meshgrid_to_mesh(terrain)
        mesh = o3d.geometry.TriangleMesh(
            o3d.utility.Vector3dVector(verts),
            o3d.utility.Vector3iVector(tris)
        )
        mesh.compute_vertex_normals()
        viz_objects.append(mesh)

        if not selection:
            data = []
            for i in range(len(tree_tuples)):
                tree_tuple = tree_tuples[i]
                data_dict = {
                    "file_name": tree_tuple["tls_tree"].clusters[0]["info"]["file"],
                    "reco_time": tree_tuple["ours_tree"].reco_time,
                    "ours_id": tree_tuple["ours_tree"].id,
                    "manual_dbh": manual_dbhs[i],
                    "tls_dbh": tls_dbhs[i],
                    "ours_dbh": ours_dbhs[i]
                }
                if tree_tuple["ours_tree"].reconstructed:
                    data_dict.update({
                        "ours_centers_x": [c.center[0] for c in tree_tuple["ours_tree"].circles],
                        "ours_centers_y": [c.center[1] for c in tree_tuple["ours_tree"].circles],
                        "ours_centers_z": [c.center[2] for c in tree_tuple["ours_tree"].circles],
                        "ours_radii": [c.radius for c in tree_tuple["ours_tree"].circles],
                    })
                else: 
                    data_dict.update({"ours_centers_x": [], "ours_centers_y": [], "ours_centers_z": [], "ours_radii": []})
                if tree_tuple["tls_tree"].reconstructed:
                    data_dict.update({
                        "tls_centers_x": [c.center[0] for c in tree_tuple["tls_tree"].circles],
                        "tls_centers_y": [c.center[1] for c in tree_tuple["tls_tree"].circles],
                        "tls_centers_z": [c.center[2] for c in tree_tuple["tls_tree"].circles], 
                        "tls_radii": [c.radius for c in tree_tuple["tls_tree"].circles],
                    })
                else:
                    data_dict.update({"tls_centers_x": [], "tls_centers_y": [], "tls_centers_z": [], "tls_radii": []})
                data.append(data_dict)
            suffixes = {"M": "mixed", "D": "deciduous", "C": "conifer"}
            if out_dir is not None:
                with open(
                    os.path.join(out_dir, 
                                f"{suffixes[data_path[-1]]}_{suffix}_{i_run}.csv"), "w+") as file:
                    csv_writer = csv.DictWriter(file, fieldnames=data[0].keys())
                    csv_writer.writeheader()
                    csv_writer.writerows(data)

            measurements = np.stack([tls_dbhs, manual_dbhs, ours_dbhs], axis=1)
            mask = ~np.isnan(measurements).any(axis=1)
            msrmnt_triplets = measurements[mask]
            sorting = np.argsort(msrmnt_triplets[:, 1])[::-1]
            msrmnt_triplets = msrmnt_triplets[sorting]
            sorted_and_measured_inds = np.arange(len(tree_tuples))[mask][sorting]
            tls_dbhs = msrmnt_triplets[:, 0]
            manual_dbhs = msrmnt_triplets[:, 1]
            ours_dbhs = msrmnt_triplets[:, 2]
            fig_bias_std, ax_bias_std = plt.subplots(figsize=(10, 5))
            ax_bias_std.plot(range(len(manual_dbhs)), manual_dbhs, label="manual", marker='o', color=MANUAL_COLOR)
            ax_bias_std.plot(range(len(tls_dbhs)), tls_dbhs, label="TLS", marker='o', color=TLS_COLOR)
            ax_bias_std.plot(range(len(ours_dbhs)), ours_dbhs, label="Ours", marker='o', color=OURS_COLOR)
            ax_bias_std.set_xticks(range(len(sorted_and_measured_inds)), sorted_and_measured_inds)
            ax_bias_std.legend()
        else:
            print(tree_tuples[selection]["ours_tree"].reconstructed)
            print(tree_tuples[selection]["tls_tree"].dbh, tree_tuples[selection]["ours_tree"].dbh) 
            o3d.visualization.draw_geometries(viz_objects)

### Evaluation

In [ ]:

from copy import deepcopy


def non_nan(arr):
    return arr[~np.isnan(arr)]


path = "/home/ori/git/digiforest_drs/trees/evaluation/accuracy"
suffix = ""
plot_types = ["mixed", "deciduous", "conifer", "all"] # "all" has to be last
num_datasets = 3


# Load Reconstruction Results
results = {}
evaluations = {}

for plot_type in plot_types:
    results[plot_type] = []
    evaluations[plot_type] = []
    if "all" in plot_types:
        plot_types_without_all = deepcopy(plot_types)
        plot_types_without_all.remove("all")
    for i in range(num_datasets):
        sub_evaluation = {}
        if plot_type != "all":
            sub_result_dbh = {
                "file_names": [],
                "reco_times": [],
                "ours_ids": [],
                "manual_dbhs": [],
                "tls_trees": [],
                "ours_trees": []
            }
            # load csv
            filename = f"{plot_type}"
            if suffix != "":
                filename += f"_{suffix}"
            filename += f"_{i}.csv"
            with open(os.path.join(path, filename), "r") as file:
                csv_reader = csv.DictReader(file)
                for row in csv_reader:
                    sub_result_dbh["file_names"].append(row["file_name"])
                    if "reco_time" in row:
                        sub_result_dbh["reco_times"].append(float(row["reco_time"]))
                    sub_result_dbh["ours_ids"].append(int(row["ours_id"]))
                    sub_result_dbh["manual_dbhs"].append(float(row["manual_dbh"]))
                    
                    tls_tree = Tree(int(row["file_name"].split("_")[1]))
                    tls_centers_x = [float(x) for x in row["tls_centers_x"].strip("[]").split(", ") if x != ""]
                    tls_centers_y = [float(y) for y in row["tls_centers_y"].strip("[]").split(", ") if y != ""]
                    tls_centers_z = [float(z) for z in row["tls_centers_z"].strip("[]").split(", ") if z != ""]
                    tls_radii = [float(r) for r in row["tls_radii"].strip("[]").split(", ") if r != ""]
                    tls_tree.circles = [Circle(np.array([x, y, z]), r) for x, y, z, r in zip(tls_centers_x, tls_centers_y, tls_centers_z, tls_radii)]
                    tls_dbh = float(row["tls_dbh"])
                    if not np.isnan(tls_dbh):
                        tls_tree.dbh = tls_dbh
                        tls_tree.reconstructed = True
                    sub_result_dbh["tls_trees"].append(tls_tree)
                    
                    ours_tree = Tree(int(row["ours_id"]))
                    ours_centers_x = [float(x) for x in row["ours_centers_x"].strip("[]").split(", ") if x != ""]
                    ours_centers_y = [float(y) for y in row["ours_centers_y"].strip("[]").split(", ") if y != ""]
                    ours_centers_z = [float(z) for z in row["ours_centers_z"].strip("[]").split(", ") if z != ""]
                    ours_radii = [float(r) for r in row["ours_radii"].strip("[]").split(", ") if r != ""]
                    ours_tree.circles = [Circle(np.array([x, y, z]), r) for x, y, z, r in zip(ours_centers_x, ours_centers_y, ours_centers_z, ours_radii)]
                    ours_dbh = float(row["ours_dbh"])
                    if not np.isnan(ours_dbh):
                        ours_tree.dbh = ours_dbh
                        ours_tree.reconstructed = True
                    sub_result_dbh["ours_trees"].append(ours_tree)
            
            # apply global shift to fine-align ours_trees with tls_trees
            errors = []
            for tls_tree, ours_tree in zip(sub_result_dbh["tls_trees"], sub_result_dbh["ours_trees"]):
                if ours_tree.reconstructed:
                    tree_errors = []
                    for c_ours in ours_tree.circles:
                        c_tls = tls_tree.evaluate_at_height(c_ours.center[2])
                        if c_tls:
                            tree_errors.append(c_ours.center - c_tls.center)
                    if len(tree_errors) > 0:
                        errors.append(np.array(tree_errors))  
            mean_errors = np.mean(np.vstack(errors), axis=0)
            for ours_tree in sub_result_dbh["ours_trees"]:
                for circle in ours_tree.circles:
                    circle.center -= mean_errors
        elif plot_type == "all":
            sub_result_dbh = {
                "file_names": [item for sublist in [results[plot][i]["file_names"] for plot in plot_types_without_all] for item in sublist],
                "reco_times": [item for sublist in [results[plot][i]["reco_times"] for plot in plot_types_without_all] for item in sublist],
                "ours_ids": [item for sublist in [results[plot][i]["ours_ids"] for plot in plot_types_without_all] for item in sublist],
                "manual_dbhs": [item for sublist in [results[plot][i]["manual_dbhs"] for plot in plot_types_without_all] for item in sublist],
                "tls_trees": [item for sublist in [results[plot][i]["tls_trees"] for plot in plot_types_without_all] for item in sublist],
                "ours_trees": [item for sublist in [results[plot][i]["ours_trees"] for plot in plot_types_without_all] for item in sublist]
            }
        # evaluate
        if plot_type != "all":
            ours_dbhs = np.array([tree.dbh  if tree.dbh else np.nan for tree in sub_result_dbh["ours_trees"]])
            tls_dbhs = np.array([tree.dbh  if tree.dbh else np.nan for tree in sub_result_dbh["tls_trees"]])
            manual_dbhs = np.array(sub_result_dbh["manual_dbhs"])
            sub_evaluation["dbh"] = {
                "diffs": {
                    "ours2tls": (ours_dbhs - tls_dbhs),
                    "tls2manual": (tls_dbhs - manual_dbhs),
                    "ours2manual": (ours_dbhs - manual_dbhs),
                },
                "ours" : ours_dbhs,
                "tls": tls_dbhs,
                "manual": manual_dbhs
            }
            ours_centers = []; ours_diameters = []; tls_centers = []; tls_diameters = []
            for tls_tree, ours_tree in zip(sub_result_dbh["tls_trees"], sub_result_dbh["ours_trees"]):
                oc = []; od = []; tc = []; td = []
                if ours_tree.reconstructed:
                    for c_ours in ours_tree.circles:
                        c_tls = tls_tree.evaluate_at_height(c_ours.center[2])
                        if c_tls:
                            tc.append(c_tls.center)
                            td.append(2 * c_tls.radius)
                            oc.append(c_ours.center)
                            od.append(2 * c_ours.radius)
                ours_centers.append(np.array(oc)); ours_diameters.append(np.array(od)); tls_centers.append(np.array(tc)); tls_diameters.append(np.array(td))
            sub_evaluation["stem"] = {
                "ours_centers" : ours_centers,
                "ours_diameters" : ours_diameters,
                "tls_centers" : tls_centers,
                "tls_diameters" : tls_diameters
            }
            sub_evaluation["height"] = {
                "ours_heights" : np.array([tree.circles[-1].center[2] - tree.circles[0].center[2]  if tree.reconstructed and len(tree.circles) > 2 else np.nan for tree in sub_result_dbh["ours_trees"]]),
                "tls_heights" : np.array([tree.circles[-1].center[2] - tree.circles[0].center[2] for tree in sub_result_dbh["tls_trees"]])
            }
        elif plot_type == "all":
            sub_evaluation["dbh"] = {
                "diffs": {
                    "ours2tls" : np.hstack([evaluations["mixed"][i]["dbh"]["diffs"]["ours2tls"], evaluations["deciduous"][i]["dbh"]["diffs"]["ours2tls"], evaluations["conifer"][i]["dbh"]["diffs"]["ours2tls"]]),
                    "tls2manual" : np.hstack([evaluations["mixed"][i]["dbh"]["diffs"]["tls2manual"], evaluations["deciduous"][i]["dbh"]["diffs"]["tls2manual"], evaluations["conifer"][i]["dbh"]["diffs"]["tls2manual"]]),
                    "ours2manual" : np.hstack([evaluations["mixed"][i]["dbh"]["diffs"]["ours2manual"], evaluations["deciduous"][i]["dbh"]["diffs"]["ours2manual"], evaluations["conifer"][i]["dbh"]["diffs"]["ours2manual"]]),
                },
                "ours" : np.hstack([evaluations["mixed"][i]["dbh"]["ours"], evaluations["deciduous"][i]["dbh"]["ours"], evaluations["conifer"][i]["dbh"]["ours"]]),
                "tls" : np.hstack([evaluations["mixed"][i]["dbh"]["tls"], evaluations["deciduous"][i]["dbh"]["tls"], evaluations["conifer"][i]["dbh"]["tls"]]),
                "manual" : np.hstack([evaluations["mixed"][i]["dbh"]["manual"], evaluations["deciduous"][i]["dbh"]["manual"], evaluations["conifer"][i]["dbh"]["manual"]])
            }
            sub_evaluation["stem"] = {
                "ours_centers" : evaluations["mixed"][i]["stem"]["ours_centers"] + evaluations["deciduous"][i]["stem"]["ours_centers"] + evaluations["conifer"][i]["stem"]["ours_centers"],
                "ours_diameters" : evaluations["mixed"][i]["stem"]["ours_diameters"] + evaluations["deciduous"][i]["stem"]["ours_diameters"] + evaluations["conifer"][i]["stem"]["ours_diameters"],
                "tls_centers" : evaluations["mixed"][i]["stem"]["tls_centers"] + evaluations["deciduous"][i]["stem"]["tls_centers"] + evaluations["conifer"][i]["stem"]["tls_centers"],
                "tls_diameters" : evaluations["mixed"][i]["stem"]["tls_diameters"] + evaluations["deciduous"][i]["stem"]["tls_diameters"] + evaluations["conifer"][i]["stem"]["tls_diameters"]
            }
            sub_evaluation["height"] = {
                "ours_heights" : np.hstack([evaluations["mixed"][i]["height"]["ours_heights"], evaluations["deciduous"][i]["height"]["ours_heights"], evaluations["conifer"][i]["height"]["ours_heights"]]),
                "tls_heights" : np.hstack([evaluations["mixed"][i]["height"]["tls_heights"], evaluations["deciduous"][i]["height"]["tls_heights"], evaluations["conifer"][i]["height"]["tls_heights"]])
            }
        
        # aggregate metrics
        sub_evaluation["dbh"]["metrics"] = {
            "RMSE_ours2tls": np.sqrt(np.mean(np.square(non_nan(sub_evaluation["dbh"]["diffs"]["ours2tls"])))),
            "RMSE_ours2manual": np.sqrt(np.mean(np.square(non_nan(sub_evaluation["dbh"]["diffs"]["ours2manual"])))),
            "RMSE_tls2manual": np.sqrt(np.mean(np.square(non_nan(sub_evaluation["dbh"]["diffs"]["tls2manual"])))),
            "bias_ours2tls": np.mean(non_nan(sub_evaluation["dbh"]["diffs"]["ours2tls"])),
            "bias_ours2manual": np.mean(non_nan(sub_evaluation["dbh"]["diffs"]["ours2manual"])),
            "bias_tls2manual": np.mean(non_nan(sub_evaluation["dbh"]["diffs"]["tls2manual"])),
            "std_ours2tls": np.std(non_nan(sub_evaluation["dbh"]["diffs"]["ours2tls"])),
            "std_ours2manual": np.std(non_nan(sub_evaluation["dbh"]["diffs"]["ours2manual"])),
            "std_tls2manual": np.std(non_nan(sub_evaluation["dbh"]["diffs"]["tls2manual"])),
            "MAPE_ours2manual": np.mean(non_nan(np.abs(sub_evaluation["dbh"]["diffs"]["ours2manual"]) / sub_evaluation["dbh"]["manual"])) * 100,
            "MAPE_tls2manual": np.mean(non_nan(np.abs(sub_evaluation["dbh"]["diffs"]["tls2manual"]) / sub_evaluation["dbh"]["manual"])) * 100,
            "MAPE_ours2tls": np.mean(non_nan(np.abs(sub_evaluation["dbh"]["diffs"]["ours2tls"]) / sub_evaluation["dbh"]["tls"])) * 100,
        }
        sub_evaluation["stem"]["metrics"] = {
            "RMSE_centers": 100 * np.sqrt(np.mean(np.square(np.hstack([np.linalg.norm(ours - tls, axis=1) for ours, tls in zip(sub_evaluation["stem"]["ours_centers"], sub_evaluation["stem"]["tls_centers"]) if (ours - tls).shape[0] > 0])))),
            "RMSE_diameters": 100 * np.sqrt(np.mean(np.square(np.hstack([ours - tls for ours, tls in zip(sub_evaluation["stem"]["ours_diameters"], sub_evaluation["stem"]["tls_diameters"])])))),
            "bias_centers": 100 * np.mean(np.hstack([np.linalg.norm(ours - tls, axis=1) for ours, tls in zip(sub_evaluation["stem"]["ours_centers"], sub_evaluation["stem"]["tls_centers"]) if (ours - tls).shape[0] > 0])),
            "bias_diameters": 100 * np.mean(np.hstack([ours - tls for ours, tls in zip(sub_evaluation["stem"]["ours_diameters"], sub_evaluation["stem"]["tls_diameters"])])),
            "std_centers": 100 * np.std(np.hstack([np.linalg.norm(ours - tls, axis=1) for ours, tls in zip(sub_evaluation["stem"]["ours_centers"], sub_evaluation["stem"]["tls_centers"]) if (ours - tls).shape[0] > 0])),
            "std_diameters": 100 * np.std(np.hstack([ours - tls for ours, tls in zip(sub_evaluation["stem"]["ours_diameters"], sub_evaluation["stem"]["tls_diameters"])])),
            "MAPE_diameters": np.mean(np.abs(np.hstack([ours - tls for ours, tls in zip(sub_evaluation["stem"]["ours_diameters"], sub_evaluation["stem"]["tls_diameters"])])) / np.hstack([tls for tls in sub_evaluation["stem"]["tls_diameters"]])) * 100,
            "RMSE_rel_diameters": np.sqrt(np.mean(np.square(np.hstack([ours - tls for ours, tls in zip(sub_evaluation["stem"]["ours_diameters"], sub_evaluation["stem"]["tls_diameters"])])) / np.hstack(tls for tls in sub_evaluation["stem"]["tls_diameters"]))) * 100,
        }
        sub_evaluation["height"]["metrics"] = {
            "mean_ours": np.mean(non_nan(sub_evaluation["height"]["ours_heights"])),
            "mean_tls": np.mean(non_nan(sub_evaluation["height"]["tls_heights"])),
        }
        sub_evaluation["timing"] = {"metrics": {"reco_time": 1000*np.mean(non_nan(np.array(sub_result_dbh["reco_times"])))}}
        
        results[plot_type].append(sub_result_dbh)
        evaluations[plot_type].append(sub_evaluation)



metric_names = [
    # "dbh/RMSE_ours2manual", "dbh/RMSE_ours2tls", "dbh/RMSE_tls2manual",
    # "",
    "dbh/RMSE_ours2manual", "dbh/bias_ours2manual", "dbh/std_ours2manual",
    # "",
    # "dbh/bias_ours2tls", "dbh/bias_ours2manual", "dbh/bias_tls2manual",
    # "",
    # "dbh/MAPE_ours2manual", "dbh/MAPE_tls2manual", "dbh/MAPE_ours2tls",
    "-",
    "stem/RMSE_centers", "stem/bias_centers", "stem/std_centers",
    "",
    "stem/RMSE_diameters", "stem/bias_diameters", "stem/std_diameters",
    "-",
    "height/mean_ours", "height/mean_tls",
    "-",
    "timing/reco_time"
]

pad_len_metric_name=max([len(metric_name) for metric_name in metric_names]) + 2
pad_len_plot_type = max([len(plot_type) for plot_type in plot_types]) + 2
header_string = f"{'metric_name':<{pad_len_metric_name}}|"
for plot_type in plot_types:
    header_string += f"{plot_type:>{pad_len_plot_type}}"
print(f"\033[1m{header_string}\033[0m")
print("-"*len(header_string))
for metric_name in metric_names:
    if metric_name == "":
        print()
        continue
    if metric_name == "-":
        print("-"*len(header_string))
        continue
    metric_first, metric_second = metric_name.split("/")
    plot_metrics = []
    string = f"{metric_name:<{pad_len_metric_name}}|"
    for plot_type in plot_types:
        string += f"{np.mean([e[metric_first]['metrics'][metric_second] for e in evaluations[plot_type]]):>{pad_len_plot_type}.2f}"
    print(string)

### Plotting

In [ ]:

import matplotlib
matplotlib.rc('text', usetex=True)
# tls manual ours

plot_config = {
    "mixed": {
        "marker": MIXED_MARKER,
        "marker_size": 20,
        "title": "Mixed Plot",
    },
    "deciduous": {
        "marker": DECIDUOUS_MARKER,
        "marker_size": 30,
        "title":"Deciduous Trees",
        },
    "conifer": {
        "marker": CONIFER_MARKER,
        "marker_size": 15,
        "title": "Conifer Trees",
    },
    "all": {
        "title":"All Trees",
    }
}

non_nan_row = lambda arr: arr[~np.isnan(arr).any(axis=1)]
data_points = {
    "mixed" : {
        "ours2tls": non_nan_row(np.vstack([evaluations["mixed"][i]["dbh"]["ours"], evaluations["mixed"][i]["dbh"]["tls"]]).T),
        "ours2manual": non_nan_row(np.vstack([evaluations["mixed"][i]["dbh"]["ours"], evaluations["mixed"][i]["dbh"]["manual"]]).T)
    },
    "deciduous" : {
        "ours2tls": non_nan_row(np.vstack([evaluations["deciduous"][i]["dbh"]["ours"], evaluations["deciduous"][i]["dbh"]["tls"]]).T),
        "ours2manual": non_nan_row(np.vstack([evaluations["deciduous"][i]["dbh"]["ours"], evaluations["deciduous"][i]["dbh"]["manual"]]).T)
    },
    "conifer" : {
        "ours2tls": non_nan_row(np.vstack([evaluations["conifer"][i]["dbh"]["ours"], evaluations["conifer"][i]["dbh"]["tls"]]).T),
        "ours2manual": non_nan_row(np.vstack([evaluations["conifer"][i]["dbh"]["ours"], evaluations["conifer"][i]["dbh"]["manual"]]).T)
    }
}
data_points["mixed"]["min_radius"] = data_points["mixed"]["ours2tls"].min()
data_points["mixed"]["max_radius"] = data_points["mixed"]["ours2tls"].max()
data_points["deciduous"]["min_radius"] = data_points["deciduous"]["ours2tls"].min()
data_points["deciduous"]["max_radius"] = data_points["deciduous"]["ours2tls"].max()
data_points["conifer"]["min_radius"] = data_points["conifer"]["ours2tls"].min()
data_points["conifer"]["max_radius"] = data_points["conifer"]["ours2tls"].max()
data_points["all"] = {
    "min_radius": np.min([data_points["mixed"]["min_radius"], data_points["deciduous"]["min_radius"], data_points["conifer"]["min_radius"]]),
    "max_radius": np.max([data_points["mixed"]["max_radius"], data_points["deciduous"]["max_radius"], data_points["conifer"]["max_radius"]]),
}

index = 2
for plot_type in ["mixed", "deciduous", "conifer", "all"]:
    fig, ax = plt.subplots(1, 1, figsize=(3.3, 3.3))
    fig.tight_layout()
    min_radius = data_points[plot_type]["min_radius"]
    max_radius = data_points[plot_type]["max_radius"]
    ax.plot([min_radius, max_radius], [min_radius, max_radius], color="#6E6E6E", linestyle='dashed', zorder=0)
    ax.set_xlim(min_radius, max_radius)
    ax.set_ylim(min_radius, max_radius)
    ax.set_xticks(np.round(np.arange(min_radius, max_radius, 5)/5, 0)*5)
    ax.set_yticks(np.round(np.arange(min_radius, max_radius, 5)/5, 0)*5)
    if plot_type == "all":
        ax.scatter(data_points["conifer"]["ours2tls"][:, 0], data_points["conifer"]["ours2tls"][:, 1], marker=plot_config["conifer"]["marker"], s=plot_config["conifer"]["marker_size"], color=TLS_COLOR, zorder=1)
        ax.scatter(data_points["conifer"]["ours2manual"][:, 0], data_points["conifer"]["ours2manual"][:, 1], marker=plot_config["conifer"]["marker"], s=plot_config["conifer"]["marker_size"], color=MANUAL_COLOR, zorder=1)
        ax.scatter(data_points["mixed"]["ours2tls"][:, 0], data_points["mixed"]["ours2tls"][:, 1], marker=plot_config["mixed"]["marker"], s=plot_config["mixed"]["marker_size"], color=TLS_COLOR, zorder=1)
        ax.scatter(data_points["mixed"]["ours2manual"][:, 0], data_points["mixed"]["ours2manual"][:, 1], marker=plot_config["mixed"]["marker"], s=plot_config["mixed"]["marker_size"], color=MANUAL_COLOR, zorder=1)
        ax.scatter(data_points["deciduous"]["ours2tls"][:, 0], data_points["deciduous"]["ours2tls"][:, 1], marker=plot_config["deciduous"]["marker"], s=plot_config["deciduous"]["marker_size"], color=TLS_COLOR, zorder=1)
        ax.scatter(data_points["deciduous"]["ours2manual"][:, 0], data_points["deciduous"]["ours2manual"][:, 1], marker=plot_config["deciduous"]["marker"], s=plot_config["deciduous"]["marker_size"], color=MANUAL_COLOR, zorder=1)
        legend_elements = [
            matplotlib.lines.Line2D([0], [0], marker=MIXED_MARKER, color='w', markerfacecolor='k', markersize=10, label='Mixed'),
            matplotlib.lines.Line2D([0], [0], marker=DECIDUOUS_MARKER, color='k', markerfacecolor='k', linestyle='', markersize=7, label='Deciduous'),
            matplotlib.lines.Line2D([0], [0], marker=CONIFER_MARKER, color='w', markerfacecolor='k', markersize=10, label='Conifer'),
            matplotlib.patches.Patch(color=TLS_COLOR, label='wrt. TLS'),
            matplotlib.patches.Patch(color=MANUAL_COLOR, label='wrt. Manual'),
        ]
        ax.legend(handles=legend_elements, loc='upper left')
    else:
        ax.scatter(data_points[plot_type]["ours2tls"][:, 0], data_points[plot_type]["ours2tls"][:, 1], marker=plot_config[plot_type]["marker"], s=plot_config[plot_type]["marker_size"], color=TLS_COLOR, zorder=1)
        ax.scatter(data_points[plot_type]["ours2manual"][:, 0], data_points[plot_type]["ours2manual"][:, 1], marker=plot_config[plot_type]["marker"], s=plot_config[plot_type]["marker_size"], color=MANUAL_COLOR, zorder=1)
    # ax.set_xlabel("Our DBH [cm]")
    # ax.set_ylabel("Reference DBH [cm]")
    ax.set_title(plot_config[plot_type]["title"])
    fig.savefig(f"/home/ori/Documents/Leonard/drs-docs-current/2024-iros-realtime-trees/pics/dbh_scatter_{plot_type}.pdf", bbox_inches='tight')


In [ ]:
o3d.visualization.draw_geometries(viz_objects)

In [ ]:
import ipywidgets as widgets
%matplotlib inline

@widgets.interact(idx=widgets.IntSlider(min=0, max=len(centers_ours)-1, step=1, value=0))
def update_slice_height(idx):
    fig, ax = plt.subplots(3, 1, figsize=(15, 8))
    print(f"Corr X: {correlation_x[idx]:.3f}, Corr Y: {correlation_y[idx]:.3f}, Corr Radius: {correlation_radius[idx]:.3f}")
    print(f"Mean Error Dist: {mean_errors_dist[idx]*100:.3f} cm, Mean Error Radius: {mean_errors_radius[idx]*100:.3f} cm")
    ax[0].plot(range(len(centers_ours[idx])), [c[0] for c in centers_ours[idx]], label="Ours", marker='o', color='r')
    ax[0].plot(range(len(centers_gt[idx])), [c[0] for c in centers_gt[idx]], label="GT", marker='o', color='g')
    ax[0].set_title("X")
    ax[0].legend()
    ax[1].plot(range(len(centers_ours[idx])), [c[1] for c in centers_ours[idx]], label="Ours", marker='o', color='r')
    ax[1].plot(range(len(centers_gt[idx])), [c[1] for c in centers_gt[idx]], label="GT", marker='o', color='g')
    ax[1].set_title("Y")
    ax[1].legend()
    ax[2].plot(range(len(radii_ours[idx])), radii_ours[idx], label="Ours", marker='o', color='r')
    ax[2].plot(range(len(radii_gt[idx])), radii_gt[idx], label="GT", marker='o', color='g')
    ax[2].set_title("Radius")
    ax[2].legend()
    # fig.show()
 

In [ ]:
o3d.visualization.draw_geometries(viz_objects)

## Experiment 02: Global Consistency

In [ ]:
import pickle, os
path = "/home/ori/git/digiforest_drs/trees/evaluation/loop_closure_multiple"

with open(os.path.join(path, "tree_manager_without_lc.pkl"), "rb") as file:
    tm_before = pickle.load(file)
with open(os.path.join(path, "tree_manager_with_lc.pkl"), "rb") as file:
    tm_after = pickle.load(file)

if os.path.exists(os.path.join(path, "tree_manager_without-with_lc_transform.txt")):
    with open(os.path.join(path, "tree_manager_without-with_lc_transform.txt"), "r") as file:
        T = []
        for row in file:
            T.append([float(x) for x in row.split()])
        T = np.array(T)
        for tree in tm_before.trees:
            tree.apply_transform(T[:3, 3], T[:3, :3])
        # for tree in tm_after.trees:
        #     tree.apply_transform(T[:3, 3], T[:3, :3])

tuples_before, terrain_interpolator = load_tuples("/home/ori/datasets/digiforest_gt_prefor_C", tm_before)
tuples_after, terrain_interpolator = load_tuples("/home/ori/datasets/digiforest_gt_prefor_C", tm_after)

In [ ]:
from tqdm.auto import tqdm
duration_threshold = 500
for tuple in tqdm(tuples_before + tuples_after):
    tuple["ours_tree"].reconstruct3(max_radius=0.5)
    tuple["ours_tree"].compute_dbh(terrain_interpolator(tuple["ours_tree"].axis["transform"][:2, 3])[0])

dbh_diffs_before = []; dbh_oldness_mask_before = []
for tuple in tuples_before:
    if "manual_dbh" not in tuple:
        continue
    if len(tuple["ours_tree"].clusters) < 2:
        continue
    if tuple["ours_tree"].dbh is not None:
        dbh_diffs_before.append(tuple["ours_tree"].dbh * 100 - tuple["manual_dbh"])
        time_stamps = [c["info"]["time_stamp"].secs for c in tuple["ours_tree"].clusters]
        dbh_oldness_mask_before.append((max(time_stamps) - min(time_stamps)) > duration_threshold)
dbh_diffs_after = []; dbh_oldness_mask_after = []
for tuple in tuples_after:
    if "manual_dbh" not in tuple:
        continue
    if len(tuple["ours_tree"].clusters) < 2:
        continue
    if tuple["ours_tree"].dbh is not None:
        dbh_diffs_after.append(tuple["ours_tree"].dbh * 100 - tuple["manual_dbh"])
        time_stamps = [c["info"]["time_stamp"].secs for c in tuple["ours_tree"].clusters]
        dbh_oldness_mask_after.append((max(time_stamps) - min(time_stamps)) > duration_threshold)

dbh_diffs_before = np.array(dbh_diffs_before); dbh_oldness_mask_before = np.array(dbh_oldness_mask_before)
dbh_diffs_after = np.array(dbh_diffs_after); dbh_oldness_mask_after = np.array(dbh_oldness_mask_after)

stem_diffs_dist_before = []; stem_diffs_dia_before = []; stem_oldness_mask_before = []
for tuple in tuples_before:
    if len(tuple["ours_tree"].clusters) < 2:
        continue
    if not tuple["ours_tree"].reconstructed:
        continue
    errors_center = []
    errors_radius = []
    for c_ours in tuple["ours_tree"].circles:
        c = tuple["tls_tree"].evaluate_at_height(c_ours.center[2])
        if c is not None:
            errors_center.append(100 * (c.center - c_ours.center))
            errors_radius.append(200 * (c.radius - c_ours.radius))
    stem_diffs_dist_before.extend([np.linalg.norm(e) for e in errors_center])
    stem_diffs_dia_before.extend(errors_radius)
    time_stamps = [c["info"]["time_stamp"].secs for c in tuple["ours_tree"].clusters]
    stem_oldness_mask_before.extend([(max(time_stamps) - min(time_stamps)) > duration_threshold] * len(errors_center))
stem_diffs_dist_before = np.array(stem_diffs_dist_before); stem_diffs_dia_before = np.array(stem_diffs_dia_before); stem_oldness_mask_before = np.array(stem_oldness_mask_before)

stem_diffs_dist_after = []; stem_diffs_dia_after = []; stem_oldness_mask_after = []
for tuple in tuples_after:
    if len(tuple["ours_tree"].clusters) < 2:
        continue
    if not tuple["ours_tree"].reconstructed:
        continue
    errors_center = []
    errors_radius = []
    for c_ours in tuple["ours_tree"].circles:
        c = tuple["tls_tree"].evaluate_at_height(c_ours.center[2])
        if c is not None:
            errors_center.append(100 * (c.center - c_ours.center))
            errors_radius.append(200 * (c.radius - c_ours.radius))
    stem_diffs_dist_after.extend([np.linalg.norm(e) for e in errors_center])
    stem_diffs_dia_after.extend(errors_radius)
    time_stamps = [c["info"]["time_stamp"].secs for c in tuple["ours_tree"].clusters]
    stem_oldness_mask_after.extend([(max(time_stamps) - min(time_stamps)) > duration_threshold] * len(errors_center))
stem_diffs_dist_after = np.array(stem_diffs_dist_after); stem_diffs_dia_after = np.array(stem_diffs_dia_after); stem_oldness_mask_after = np.array(stem_oldness_mask_after)

print("ALL TREES")
print(f"DBH Before       | {np.sqrt(np.mean(np.square(dbh_diffs_before))):>10.3f} {np.mean(dbh_diffs_before):>10.3f} {np.std(dbh_diffs_before):>10.3f}")
print(f"DBH After        | {np.sqrt(np.mean(np.square(dbh_diffs_after))):>10.3f} {np.mean(dbh_diffs_after):>10.3f} {np.std(dbh_diffs_after):>10.3f}")
print("")
print(f"Stem Dist Before | {np.sqrt(np.mean(np.square(stem_diffs_dist_before))):>10.3f} {np.mean(stem_diffs_dist_before):>10.3f} {np.std(stem_diffs_dist_before):>10.3f}")
print(f"Stem Dist After  | {np.sqrt(np.mean(np.square(stem_diffs_dist_after))):>10.3f} {np.mean(stem_diffs_dist_after):>10.3f} {np.std(stem_diffs_dist_after):>10.3f}")
print("")
print(f"Stem Rad Before  | {np.sqrt(np.mean(np.square(stem_diffs_dia_before))):>10.3f} {np.mean(stem_diffs_dia_before):>10.3f} {np.std(stem_diffs_dia_before):>10.3f}")
print(f"Stem Rad After   | {np.sqrt(np.mean(np.square(stem_diffs_dia_after))):>10.3f} {np.mean(stem_diffs_dia_after):>10.3f} {np.std(stem_diffs_dia_after):>10.3f}")

dbh_diffs_before = dbh_diffs_before[dbh_oldness_mask_before]
dbh_diffs_after = dbh_diffs_after[dbh_oldness_mask_after]
stem_diffs_dist_before = stem_diffs_dist_before[stem_oldness_mask_before]
stem_diffs_dist_after = stem_diffs_dist_after[stem_oldness_mask_after]
stem_diffs_dia_before = stem_diffs_dia_before[stem_oldness_mask_before]
stem_diffs_dia_after = stem_diffs_dia_after[stem_oldness_mask_after]
print()
print("OLD TREES")
print(f"DBH Before       | {np.sqrt(np.mean(np.square(dbh_diffs_before))):>10.3f} {np.mean(dbh_diffs_before):>10.3f} {np.std(dbh_diffs_before):>10.3f}")
print(f"DBH After        | {np.sqrt(np.mean(np.square(dbh_diffs_after))):>10.3f} {np.mean(dbh_diffs_after):>10.3f} {np.std(dbh_diffs_after):>10.3f}")
print("")
print(f"Stem Dist Before | {np.sqrt(np.mean(np.square(stem_diffs_dist_before))):>10.3f} {np.mean(stem_diffs_dist_before):>10.3f} {np.std(stem_diffs_dist_before):>10.3f}")
print(f"Stem Dist After  | {np.sqrt(np.mean(np.square(stem_diffs_dist_after))):>10.3f} {np.mean(stem_diffs_dist_after):>10.3f} {np.std(stem_diffs_dist_after):>10.3f}")
print("")
print(f"Stem Rad Before  | {np.sqrt(np.mean(np.square(stem_diffs_dia_before))):>10.3f} {np.mean(stem_diffs_dia_before):>10.3f} {np.std(stem_diffs_dia_before):>10.3f}")
print(f"Stem Rad After   | {np.sqrt(np.mean(np.square(stem_diffs_dia_after))):>10.3f} {np.mean(stem_diffs_dia_after):>10.3f} {np.std(stem_diffs_dia_after):>10.3f}")


In [ ]:
ages = []
for tuple in tuples_before:
    time_stamps = [c["info"]["time_stamp"].secs for c in tuple["ours_tree"].clusters]
    ages.append(max(time_stamps) - min(time_stamps))
print(np.sort(ages))

In [ ]:
index =  21
slice_height = 4.2; slice_width = 0.2
tree_before = tuples_before[index]["ours_tree"]
for tuple in tuples_after:
    if np.linalg.norm(tuple["ours_tree"].axis["transform"][:3, 3] - tree_before.axis["transform"][:3, 3]) < 0.5:
        tree_after = tuple["ours_tree"]
        break

tree_before_points = [c["cloud"].point.positions.numpy() @ c["info"]["T_sensor2map"][:3, :3].T + c["info"]["T_sensor2map"][:3, 3] for c in tree_before.clusters]
tree_after_points = [c["cloud"].point.positions.numpy() @ c["info"]["T_sensor2map"][:3, :3].T + c["info"]["T_sensor2map"][:3, 3] for c in tree_after.clusters]

slice_height_before = terrain_interpolator(tree_before.axis["transform"][:2, 3])[0] + slice_height
slice_height_after = terrain_interpolator(tree_after.axis["transform"][:2, 3])[0] + slice_height
tree_before_slice = [c[np.logical_and(c[:, 2] > slice_height_before - slice_width / 2, c[:, 2] < slice_height_before + slice_width / 2)] for c in tree_before_points]
tree_after_slice = [c[np.logical_and(c[:, 2] > slice_height_after - slice_width / 2, c[:, 2] < slice_height_after + slice_width / 2)] for c in tree_after_points]



import matplotlib
matplotlib.rc('text', usetex=True)  
from matplotlib import pyplot as plt
import colorsys
all_points_before = np.vstack(tree_before_slice)
xlims = [np.min(all_points_before[:, 0]), np.max(all_points_before[:, 0])]
ylims = [np.min(all_points_before[:, 1]), np.max(all_points_before[:, 1])]

fig_before, ax_before = plt.subplots(1, 1, figsize=(2, 2))
fig_before.tight_layout()
ax_before.set_aspect('equal')
ax_before.set_xlim(*xlims)
ax_before.set_ylim(*ylims)
ax_before.axis('off')
# ax_before.set_title("Without Loop Closure")
hues = np.linspace(0, 1, len(tree_before_slice) + 1)[:-1]
for c, h in zip(tree_before_slice, hues):
    ax_before.scatter(c[:, 0], c[:, 1], s=0.8, color=colorsys.hls_to_rgb(h, 0.45, 1))
# fig_before.savefig("/home/ori/Documents/Leonard/drs-docs-current/2024-iros-realtime-trees/pics/cloud_1_without_loop_closure.pdf", bbox_inches='tight')

fig_after, ax_after = plt.subplots(1, 1, figsize=(2, 2))
fig_after.tight_layout()
ax_after.set_aspect('equal')
ax_after.axis('off')
ax_after.set_xlim(*xlims)
ax_after.set_ylim(*ylims)
# ax_after.set_title("With Loop Closure")
for c, h in zip(tree_after_slice, hues):
    ax_after.scatter(c[:, 0], c[:, 1], s=0.8, color=colorsys.hls_to_rgb(h, 0.45, 1))
# fig_after.savefig("/home/ori/Documents/Leonard/drs-docs-current/2024-iros-realtime-trees/pics/cloud_1_with_loop_closure.pdf", bbox_inches='tight')

## Experiment 03: Ablations

### Hough

In [ ]:
import ipywidgets as widgets
from matplotlib import pyplot as plt
import colorsys
import matplotlib
%matplotlib inline
# matplotlib.rc('text', usetex=False)

# slice_height = 1.85; slice_width = 0.2; index = 9 # hough overestimates
# slice_height = 4.3; slice_width = 0.2; index = 22, i_cluster = 2 # ransac overestimates
# slice_height = 4.3; slice_width = 0.2; index = 22, i_cluster = 2 # ransac overestimates
index = 28
slice_height = 4.3
slice_width = 0.2

fig.tight_layout()
def update_plot(slice_height, index, i_cluster):
    fig, ax = plt.subplots(1, 1, figsize=(5, 5))
    tree = datasets["conifer"]["tuples"][index]["ours_tree"]
    tls_circle = datasets["conifer"]["tuples"][index]["tls_tree"].evaluate_at_height(slice_height)
    hues = np.linspace(0, 1, len(tree.clusters) + 1)[:-1]
    ax.set_aspect('equal')
    ax.set_xlim(tls_circle.center[0] - 3 * tls_circle.radius, tls_circle.center[0] + 3 * tls_circle.radius)
    ax.set_ylim(tls_circle.center[1] - 3 * tls_circle.radius, tls_circle.center[1] + 3 * tls_circle.radius)
    # ax.set_ylim(-7, -5.5)
    # ax.axis('off')
    for i in range(len(tree.clusters)):
    # for i in range(len(tree.clusters))[i_cluster:i_cluster+1]:
        color = colorsys.hls_to_rgb(hues[i], 0.45, 1)
        slice_points = tree.clusters[i]["cloud"].point.positions.numpy() @ tree.clusters[i]["info"]["T_sensor2map"][:3, :3].T + tree.clusters[i]["info"]["T_sensor2map"][:3, 3]
        slice_points = slice_points[np.logical_and(slice_points[:, 2] > slice_height - slice_width / 2, slice_points[:, 2] < slice_height + slice_width / 2)]
        if len(slice_points) == 0:
            continue    
        
        # hough_code = '''circ_hough = Circle.from_cloud_hough(slice_points, max_radius=0.5)'''
        # ransac_code = '''circ_ransac = Circle.from_cloud_ransac(slice_points, max_radius=0.5)'''
        # ransahc_code = '''circ_ransahc = Circle.from_cloud_ransahc(slice_points, max_radius=0.5)'''
        # # print("Hough Time:", timeit.timeit(stmt=hough_code, globals=globals(), number=100))
        # # print("RANSAC Time:", timeit.timeit(stmt=ransac_code, globals=globals(), number=100))
        # # print("RANSAHC Time:", timeit.timeit(stmt=ransahc_code, globals=globals(), number=100))
        # exec(hough_code)
        # # exec(ransac_code)
        # exec(ransahc_code)
        circ_hough = Circle.from_cloud_hough(slice_points, max_radius=0.5)
        circ_ransac = Circle.from_cloud_ransac(slice_points, max_radius=0.5)
        circ_ransahc = Circle.from_cloud_ransahc(slice_points, max_radius=0.5)

        ax.scatter(slice_points[:, 0], slice_points[:, 1], s=0.8, color="k")
        if circ_hough is not None:
            ax.add_artist(plt.Circle(circ_hough.center[:2], circ_hough.radius, fill=False, color='r', label="Hough"))
        if circ_ransac is not None:
            ax.add_artist(plt.Circle(circ_ransac.center[:2], circ_ransac.radius, fill=False, color='g', label="RANSAC"))
        if circ_ransahc is not None:
            ax.add_artist(plt.Circle(circ_ransahc.center[:2], circ_ransahc.radius, fill=False, color='b', label="RANSAHC"))
    legend_elements = [
        matplotlib.patches.Patch(label="Hough", fill=None, edgecolor='r'),
        matplotlib.patches.Patch(label="RANSAC", fill=None, edgecolor='g'),
        matplotlib.patches.Patch(label="RANSAHC", fill=None, edgecolor='b'),
        matplotlib.patches.Patch(label="TLS", fill=None, edgecolor='k'),
    ]
    ax.legend(handles=legend_elements, loc='lower right')
    ax.add_artist(plt.Circle(tls_circle.center[:2], tls_circle.radius, fill=False, color="y", linestyle='dashed', label="TLS", linewidth=2, zorder=1))
    plt.show()

slice_height_slider = widgets.FloatSlider(value=slice_height, min=0, max=10, step=0.01, description='Slice Height:')
index_slider = widgets.IntSlider(value=index, min=0, max=len(datasets["conifer"]["tuples"])-1, step=1, description='Index:')
cluster_slider = widgets.IntSlider(value=0, min=0, max=len(datasets["conifer"]["tuples"][index]["ours_tree"].clusters)-1, step=1, description='Cluster:')
# save_button = widgets.Button(description='Save Graphic')
# def save_graphic(button):
#     fig.savefig('/home/ori/Documents/Leonard/drs-docs-current/2024-iros-realtime-trees/pics/ablation_#.pdf')
# save_button.on_click(save_graphic)
# display(save_button)
widgets.interact(update_plot, slice_height=slice_height_slider, index=index_slider, i_cluster=cluster_slider)


In [ ]:
import ipywidgets as widgets
from matplotlib import pyplot as plt
import colorsys
import matplotlib
%matplotlib inline
# matplotlib.rc('text', usetex=False)

# index = 28; slice_height = 4.3; slice_width = 0.2
index = 17; slice_height = 3.9; slice_width = 0.2

fig.tight_layout()
fig_ransahc, ax_ransahc = plt.subplots(1, 1, figsize=(2.5, 2.5))
fig_ransac, ax_ransac = plt.subplots(1, 1, figsize=(2.5, 2.5))
fig_hough, ax_hough = plt.subplots(1, 1, figsize=(2.5, 2.5))

tree = datasets["conifer"]["tuples"][index]["ours_tree"]
tls_circle = datasets["conifer"]["tuples"][index]["tls_tree"].evaluate_at_height(slice_height)
for ax in [ax_ransahc, ax_ransac, ax_hough]:
    ax.set_aspect('equal')
    ax.set_xlim(tls_circle.center[0] - 2 * tls_circle.radius, tls_circle.center[0] + 2 * tls_circle.radius)
    ax.set_ylim(tls_circle.center[1] - 2 * tls_circle.radius, tls_circle.center[1] + 2 * tls_circle.radius)
    ax.axis('off')
    ax.add_artist(plt.Circle(tls_circle.center[:2], tls_circle.radius, fill=False, color=TLS_COLOR, linestyle='dashed', label="TLS", linewidth=3, zorder=2))
    
for i in range(len(tree.clusters)):
# for i in range(len(tree.clusters))[i_cluster:i_cluster+1]:
    slice_points = tree.clusters[i]["cloud"].point.positions.numpy() @ tree.clusters[i]["info"]["T_sensor2map"][:3, :3].T + tree.clusters[i]["info"]["T_sensor2map"][:3, 3]
    slice_points = slice_points[np.logical_and(slice_points[:, 2] > slice_height - slice_width / 2, slice_points[:, 2] < slice_height + slice_width / 2)]
    if len(slice_points) == 0:
        continue    
    
    # hough_code = '''circ_hough = Circle.from_cloud_hough(slice_points, max_radius=0.5)'''
    # ransac_code = '''circ_ransac = Circle.from_cloud_ransac(slice_points, max_radius=0.5)'''
    # ransahc_code = '''circ_ransahc = Circle.from_cloud_ransahc(slice_points, max_radius=0.5)'''
    # # print("Hough Time:", timeit.timeit(stmt=hough_code, globals=globals(), number=100))
    # # print("RANSAC Time:", timeit.timeit(stmt=ransac_code, globals=globals(), number=100))
    # # print("RANSAHC Time:", timeit.timeit(stmt=ransahc_code, globals=globals(), number=100))
    # exec(hough_code)
    # # exec(ransac_code)
    # exec(ransahc_code)
    circ_hough = Circle.from_cloud_hough(slice_points, max_radius=0.5)
    circ_ransac = Circle.from_cloud_ransac(slice_points, max_radius=0.5)
    circ_ransahc = Circle.from_cloud_ransahc(slice_points, max_radius=0.5)

    for ax in [ax_hough, ax_ransac, ax_ransahc]:
        ax.scatter(slice_points[:, 0], slice_points[:, 1], s=0.8, color="k")
    if circ_hough is not None:
        ax_hough.add_artist(plt.Circle(circ_hough.center[:2], circ_hough.radius, fill=False, color='r', label="Hough", zorder=1))
    if circ_ransac is not None:
        ax_ransac.add_artist(plt.Circle(circ_ransac.center[:2], circ_ransac.radius, fill=False, color='g', label="RANSAC", zorder=1))
    if circ_ransahc is not None:
        ax_ransahc.add_artist(plt.Circle(circ_ransahc.center[:2], circ_ransahc.radius, fill=False, color='b', label="RANSAHC", zorder=1))
legend_elements = [
    matplotlib.patches.Patch(label="Hough", fill=None, edgecolor='r'),
    matplotlib.patches.Patch(label="RANSAC", fill=None, edgecolor='g'),
    matplotlib.patches.Patch(label="RANSAHC", fill=None, edgecolor='b'),
    matplotlib.patches.Patch(label="TLS", fill=None, edgecolor='k'),
]
# ax.legend(handles=legend_elements, loc='lower right')
fig_ransahc.savefig("/home/ori/Documents/Leonard/drs-docs-current/2024-iros-realtime-trees/pics/ablation_ransahc_2.pdf", bbox_inches='tight')
fig_ransac.savefig("/home/ori/Documents/Leonard/drs-docs-current/2024-iros-realtime-trees/pics/ablation_ransac_2.pdf", bbox_inches='tight')
fig_hough.savefig("/home/ori/Documents/Leonard/drs-docs-current/2024-iros-realtime-trees/pics/ablation_hough_2.pdf", bbox_inches='tight')

### Coverage Angle

In [ ]:
from tqdm.auto import tqdm
results_dbh = []
results_stem = []

max_radii = {"conifer": 0.5, "deciduous": 0.15, "mixed": 0.25}
for plot_type, dataset in datasets.items():
    tree_tuples = dataset["tuples"]
    terrain_interpolator = dataset["terrain"]["interpolator"]
    for tree_tuple in tqdm(tree_tuples):
        ground_elevation = terrain_interpolator(tree_tuple["ours_tree"].axis["transform"][:2, 3])[0]
        tree_tuple["tls_tree"].compute_dbh(ground_elevation)
        sub_result_dbh = {"tls_dbh" : tree_tuple["tls_tree"].dbh * 100 if tree_tuple["tls_tree"].dbh is not None else np.nan, "manual_dbh" : tree_tuple["manual_dbh"] if "manual_dbh" in tree_tuple else np.nan, "ours_dbh": []}
        ours_tree : Tree = tree_tuple["ours_tree"]
        ours_tree_clusters = ours_tree.clusters
        cluster_coverage_angles = np.array([(c["info"]["coverage"]["angle_from"], c["info"]["coverage"]["angle_to"]) for c in ours_tree_clusters])
        sort_indices = np.argsort([angle_to - angle_from for angle_from, angle_to in cluster_coverage_angles])
        for i_sort in range(1, len(sort_indices)):
            ours_tree.clusters = [ours_tree_clusters[i_cluster] for i_cluster in sort_indices[:i_sort]]
            coverage_angle = tm.compute_angle_coverage(cluster_coverage_angles[sort_indices[:i_sort]])
            ours_tree.dbh = None
            reco_success = ours_tree.reconstruct3(max_radius=max_radii[plot_type])
            ours_tree.compute_dbh(ground_elevation)
            if reco_success and ours_tree.dbh is not None:
                reference = tree_tuple["manual_dbh"] if "manual_dbh" in tree_tuple else tree_tuple["tls_tree"].dbh * 100 if tree_tuple["tls_tree"].dbh is not None else np.nan
                sub_result_dbh["ours_dbh"].append({"coverage_angle" : coverage_angle, "dbh" : ours_tree.dbh * 100})
            else:
                sub_result_dbh["ours_dbh"].append({"coverage_angle" : coverage_angle, "dbh" : np.nan})
            if reco_success:
                results_stem.append({"coverage": [], "distances": [], "radii": []})
                for c_ours in ours_tree.circles:
                    c = tree_tuple["tls_tree"].evaluate_at_height(c_ours.center[2])
                    if c is not None:
                        results_stem[-1]["coverage"].append(coverage_angle)
                        results_stem[-1]["distances"].append(100 * np.linalg.norm(c.center - c_ours.center))
                        results_stem[-1]["radii"].append(100 * (c.radius - c_ours.radius))
                
        ours_tree.clusters = ours_tree_clusters
        results_dbh.append(sub_result_dbh)

In [ ]:
from matplotlib import pyplot as plt
from scipy.interpolate import make_interp_spline
import matplotlib
matplotlib.rc('text', usetex=True)

def smoothify(x, *y):
    xnew = np.linspace(x.min(), x.max(), 1000)
    y_new = []
    for y_i in y:
        spl = make_interp_spline(x, y_i, k=3)
        y_new.append(spl(xnew))
    return xnew, *y_new
    
# dbh
color = MANUAL_COLOR
coverage_angles = []
errors_tls = []
errors_manual = []
for result in results_dbh:
    coverage_angles.extend([np.rad2deg(r["coverage_angle"]) for r in result["ours_dbh"]])
    errors_tls.extend([r["dbh"] - result["tls_dbh"] for r in result["ours_dbh"]])
    errors_manual.extend([r["dbh"] - result["manual_dbh"] for r in result["ours_dbh"]])
coverage_angles = np.array(coverage_angles)
errors_tls = np.array(errors_tls)
mask_tls = ~np.isnan(np.vstack([coverage_angles, errors_tls])).any(axis=0)
mask_tls = np.logical_and(mask_tls, errors_tls < 10)
errors_tls = errors_tls[mask_tls]
coverage_angles_tls = coverage_angles[mask_tls]
errors_manual = np.array(errors_manual)
mask_manual = ~np.isnan(np.vstack([coverage_angles, errors_manual])).any(axis=0)    
mask_manual = np.logical_and(mask_manual, errors_manual < 10)
errors_manual = errors_manual[mask_manual]
coverage_angles_manual = coverage_angles[mask_manual]
coverage_angles = coverage_angles_manual
errors = errors_manual

# stem
# color = TLS_COLOR
# coverage_angles = np.array([np.rad2deg(ca) for result in results_stem for ca in result["coverage"]])
# # errors = np.array([d for result in results_stem for d in result["distances"]])
# errors = np.array([2*r for result in results_stem for r in result["radii"]])
# mask = np.abs(errors) < 10 # filter outliers
# coverage_angles = coverage_angles[mask]
# errors = errors[mask]


means = []; stds = []; rmses = []
num_buckets = 18
bucket_boundaries = np.linspace(0, 360, num_buckets + 1, endpoint=True)
bucket_centers = (bucket_boundaries[1:] + bucket_boundaries[:-1]) / 2
bucket_mask = np.zeros(len(bucket_centers), dtype=bool)
for i in range(len(bucket_boundaries) - 1):
    mask = np.logical_and(coverage_angles > bucket_boundaries[i], coverage_angles <= bucket_boundaries[i + 1])
    if mask.sum() > 3:
        bucket = errors[mask]
        means.append(np.mean(bucket))
        rmses.append(np.sqrt(np.mean(bucket ** 2)))
        stds.append(np.std(bucket))
        bucket_mask[i] = True
        
bucket_centers = bucket_centers[bucket_mask]
means = np.array(means); stds = np.array(stds); rmses = np.array(rmses)

median_coverage_angle = np.median([c["info"]["coverage"]["angle_to"] - c["info"]["coverage"]["angle_from"] for dataset in datasets.values() for t in dataset["tuples"] for c in t["ours_tree"].clusters])
print(np.rad2deg(median_coverage_angle))

fig_rmse, ax_rmse = plt.subplots(1, 1, figsize=(4, 1.5))
ax_rmse.bar(bucket_centers, rmses, width=360/num_buckets, label="Manual", color=color, alpha=0.5, edgecolor=color)
# ax_rmse.plot(bucket_centers, rmses, label="Manual", marker="s", color=color)
# ax_rmse.scatter(bucket_centers, rmses, label="Manual", marker="s", s=20, color=color)
# c = np.linalg.lstsq(np.vstack([bucket_centers, np.ones(len(bucket_centers))]).T, rmses, rcond=None)[0] # fit lsq line to rmses
# ax_rmse.plot([0, 360], [c[1], c[0] * 360 + c[1]], color=color, linestyle='dashed')
# ax_rmse.scatter(coverage_angles_manual, errors_manual, color=color, s=1)
ax_rmse.set_xlabel("Coverage Angle [deg]")
ax_rmse.set_ylabel("RMSE [cm]")
ax_rmse.set_xticks([0, 90, 180, 270, 360])
ax_rmse.set_xlim(0, 360)
# ax_rmse.legend()
# fig_rmse.savefig("/home/ori/Documents/Leonard/drs-docs-current/2024-iros-realtime-trees/pics/coverage_ablation_rmse.pdf", bbox_inches='tight')


fig_bias_std, ax_bias_std = plt.subplots(1, 1, figsize=(4, 1.5))
ax_bias_std.plot([0, 360], [0, 0], color="#6E6E6E", linestyle='dashed', zorder=0)
bucket_centers, means, stds = smoothify(bucket_centers, means, stds)
ax_bias_std.plot(bucket_centers, means, label="Manual", color=color)
ax_bias_std.fill_between(bucket_centers, means - stds, means + stds, color=color, alpha=0.3)
ax_bias_std.set_xlabel("Coverage Angle [deg]")
ax_bias_std.set_ylabel("Error in DBH [cm]")
ax_bias_std.set_xticks([0, 90, 180, 270, 360])
ax_bias_std.set_xlim(0, 360)
# fig_bias_std.savefig("/home/ori/Documents/Leonard/drs-docs-current/2024-iros-realtime-trees/pics/coverage_ablation_bias_std.pdf", bbox_inches='tight')

In [ ]:
import pickle
import sys
filename = "/home/ori/datasets/digiforest_gt_prefor_M/tree_manager.pkl"
with open(filename, "rb") as file:
    tm = pickle.load(file)



for tree in tm.trees:
    for cluster in tree.clusters:
        del cluster["cloud"]

with open(filename.replace(".pkl", "_no_points.pkl"), "wb") as file:
    pickle.dump(tm, file)

In [ ]:
tm.trees[0].clusters[0]["cloud"]

## Experiment 04: Timing

In [ ]:
plot_types = ["coniferous", "deciduous", "mixed"]
tree_managers = {}
results = {}
for plot_type in plot_types:
    with open(f"/home/ori/git/digiforest_drs/trees/evaluation/timing/tree_manager_{plot_type}.pkl", "rb") as file:
        tree_managers[plot_type] = pickle.load(file)
    sub_result_dbh = {}

    timings = [{**tr["cw"]._events, **tr["cwc"]._events} for tr in tree_managers[plot_type].timing_results]

    unique_time_stamps = np.sort(list(set([c["info"]["time_stamp"].secs for tree in tree_managers[plot_type].trees for c in tree.clusters ])))
    # time_stamp_tree_counts = np.sort([np.sum([time_stamp in [c["info"]["time_stamp"].secs for c in tree.clusters] for tree in tree_managers[plot_type].trees]) for time_stamp in unique_time_stamps])

    time_stamp_tree_counts = []
    for i in range(len(unique_time_stamps)):
        count = 0
        for tree in tree_managers[plot_type].trees:
            for ts in unique_time_stamps[:i]:
                if ts in [c["info"]["time_stamp"].secs for c in tree.clusters]:
                    count += 1
                    break
        time_stamp_tree_counts.append(count)

    for key in timings[0].keys():
        if key in ["cw", "cwc", "cwc/publishing_clustering"]:
            continue
        times = [t[key]['time'] for t in timings[1:]]
        # print(f"{key:<30}: {np.mean(times):.3f} \\pm {np.std(times):.3f} s")
    all_times = np.array([t["cw"]["time"] + t["cwc"]["time"] for t in timings])
    # print(f"{'Total':<30}: {np.mean(all_times):.3f} \\pm {np.std(all_times):.3f} s", "\n")

    durations = np.diff(unique_time_stamps)
    results[plot_type] = {
        "unique_time_stamps" : unique_time_stamps,
        "time_stamp_tree_counts" : time_stamp_tree_counts,
        "durations" : durations,
        "all_times" : all_times,
        "timings" : timings
    }

fig, ax = plt.subplots(1, 1, figsize=(3.5,1.5))

plot_names = {"coniferous": "Conifer", "deciduous": "Deciduous", "mixed": "Mixed"}
names = {"cw/terrain": "Terrain", "cw/clustering": "Clustering", "cwc/tree_manager": "Tree Manager", "cwc/publishing_tree_manager": "Visualization", "cw/conversion": "Misc"}
colors = {"cw/terrain": "#56b4e9", "cw/clustering": "#d55e00", "cwc/tree_manager": "#0173b2", "cwc/publishing_tree_manager": "#ece133", "cw/conversion": "#949494"}
payload_time_stamps = list(set([c["info"]["time_stamp"].secs for plot_type in plot_types for tree in tree_managers[plot_type].trees for c in tree.clusters]))
payload_ages = np.diff(np.sort(payload_time_stamps))
payload_ages = payload_ages[payload_ages < 1000]
mean_payload_age = np.mean(payload_ages)

for i_pt, plot_type in enumerate(plot_types):
    keys = [key for key in results[plot_type]["timings"][0].keys() if key not in ["cw", "cwc", "cwc/publishing_clustering"]]
    keys = ["cw/terrain", "cw/clustering", "cwc/tree_manager", "cwc/publishing_tree_manager", "cw/conversion"]
    runtimes = [np.mean([t[key]["time"] for t in results[plot_type]["timings"]]) for key in keys]
    for i, (k, r) in enumerate(zip(keys, runtimes)):
        # if k == "cw/conversion":
        #     continue
        ax.barh(len(plot_types) - i_pt, r, label=names[k], left=np.sum(runtimes[:i]), color=colors[k])
    ax.errorbar(np.sum(runtimes), len(plot_types) - i_pt, xerr=np.std(results[plot_type]["all_times"]), color="k", capsize=6, fmt=".")    
ax.set_yticklabels([""] + [plot_names[p] for p in plot_types][::-1]) 
ax.set_xlabel("Time [s]")
ax.plot([mean_payload_age, mean_payload_age], [0.5, len(plot_types) + 0.5], color="k", linestyle="dashed", label="Mean Payload\nCollection Time")
handles, labels = ax.get_legend_handles_labels()
by_label = dict(zip(labels, handles))
ax.legend(by_label.values(), by_label.keys(), ncol=1, bbox_to_anchor=(1.0, 1.05))
all_all_times = np.concatenate([results[plot_type]["all_times"] for plot_type in plot_types])
fig.savefig("/home/ori/Documents/Leonard/drs-docs-current/2024-iros-realtime-trees/pics/timings.pdf", bbox_inches='tight')
# print("all")
# print (f"{'Total':<30}: {np.mean(all_all_times):.3f} \\pm {np.std(all_all_times):.3f} s", "\n")

In [ ]:
import ipywidgets as widgets
from IPython.display import display
import matplotlib
matplotlib.rc('text', usetex=True)
from scipy.spatial import cKDTree
# Define the sliders
i_cluster_slider = widgets.IntSlider(min=0, max=len(tree.clusters)-1, step=1, value=11, description='Cluster Index:')
i_tree_slider = widgets.IntSlider(min=0, max=len(datasets["conifer"]["tuples"])-1, step=1, value=3, description='Tree Index:')
slice_height_slider = widgets.FloatSlider(min=0, max=1, step=0.1, value=slice_height, description='Slice Height:')
slice_width_slider = widgets.FloatSlider(min=0, max=1, step=0.1, value=slice_width, description='Slice Width:')

# Define the update function
# make 3d plot
def update_plot(i_cluster, i_tree, slice_height, slice_width):
    fig, ax = plt.subplots(1, 1, figsize=(3, 2.5), subplot_kw={'projection':'3d'})
    fig.tight_layout()
    tree = datasets["conifer"]["tuples"][i_tree]["ours_tree"]
    slice_points = []
    # for i_cluster in range(len(tree.clusters))[2:5]:
    for i_cluster in [i_cluster]:
        sp = tree.clusters[i_cluster]["cloud"].point.positions.numpy() @ tree.clusters[i_cluster]["info"]["T_sensor2map"][:3, :3].T + tree.clusters[i_cluster]["info"]["T_sensor2map"][:3, 3]
        sp = sp[np.logical_and(sp[:, 2] > slice_height - slice_width / 2, sp[:, 2] < slice_height + slice_width / 2)]
        slice_points.append(sp)
    slice_points = np.vstack(slice_points)[:, :2]
    circle_actual = Circle.from_cloud_ransahc(slice_points, max_radius=0.5)

    indices = ((1 - np.random.rand(2000, len(slice_points)))).argpartition(3, axis=1)[:, :3]
    circles = Circle.from_3_2d_points(slice_points[indices])
    
    circles = circles[np.logical_and(
        np.linalg.norm(circles[:, :2] - circle_actual.center[:2], axis=1) < 0.3,
        np.abs(circles[:, 2] - circle_actual.radius) < 0.5
    )]
    
    color_mask = np.zeros(len(circles), dtype=bool)
    query_circles = circles.copy()
    query_circles[:, 2] *= 2
    kdtree = cKDTree(query_circles)
    neighbors = kdtree.query_ball_tree(kdtree, r=0.05)
    best_index = neighbors.index(max(neighbors, key=len))
    best_circle_inds = neighbors[best_index] + [best_index]
    color_mask[best_circle_inds] = True
    
    ax.grid(alpha=0.01)
    ax.set_box_aspect((np.ptp(circles[:, 0]), np.ptp(circles[:, 1]), 1.5*np.ptp(circles[:, 2])))
    ax.set_xticks(np.round(np.arange(np.min(circles[:, 0]), np.max(circles[:, 0]), 0.2), 1))
    ax.set_yticks(np.round(np.arange(np.min(circles[:, 1]), np.max(circles[:, 1]), 0.2), 1))
    ax.set_zticks(np.round(np.arange(np.min(circles[:, 2]), np.max(circles[:, 2]), 0.2), 1))
    ax.tick_params(axis='x', which='major', labelsize=6, pad=-5)
    ax.tick_params(axis='y', which='major', labelsize=6, pad=-5)
    ax.tick_params(axis='z', which='major', labelsize=6, pad=0)
    ax.set_xlabel("y",labelpad=-13); ax.set_ylabel("x",labelpad=-11); ax.set_zlabel("r",labelpad=0)
    
    ax.xaxis.pane.fill = False
    ax.yaxis.pane.fill = False
    ax.zaxis.pane.fill = False
    colors = ["r" if cm else "k" for cm in color_mask]
    ax.scatter(circles[:, 0], circles[:, 1], circles[:, 2], s=0.5, color=colors)
    # change view angle
    ax.view_init(10, -60)
    # fig.savefig("/home/ori/Documents/Leonard/drs-docs-current/2024-iros-realtime-trees/pics/hough_scatter.pdf", bbox_inches='tight')

# Display the sliders and plot
display(i_cluster_slider, i_tree_slider, slice_height_slider, slice_width_slider)
update_plot(i_cluster_slider.value, i_tree_slider.value, slice_height_slider.value, slice_width_slider.value)

# Define the slider callbacks
def i_cluster_slider_callback(change):
    update_plot(change.new, i_tree_slider.value, slice_height_slider.value, slice_width_slider.value)

def i_tree_slider_callback(change):
    update_plot(i_cluster_slider.value, change.new, slice_height_slider.value, slice_width_slider.value)

def slice_height_slider_callback(change):
    update_plot(i_cluster_slider.value, i_tree_slider.value, change.new, slice_width_slider.value)

def slice_width_slider_callback(change):
    update_plot(i_cluster_slider.value, i_tree_slider.value, slice_height_slider.value, change.new)

# Attach the callbacks to the sliders
i_cluster_slider.observe(i_cluster_slider_callback, names='value')
i_tree_slider.observe(i_tree_slider_callback, names='value')
slice_height_slider.observe(slice_height_slider_callback, names='value')
slice_width_slider.observe(slice_width_slider_callback, names='value')

